# Digital handheld multimeter (DMM) **Benning MM12**

The aim of this notebook is to introduce a possibility to communicate in Python with the Digital handheld multimeter (DMM) **Benning MM12** via the supplied infrared USB cable.

![Front view of the Benning MM12](images/Benning_MM12_front_view_small.jpeg)  
Front view of the Benning MM12

The technical data of the multimeter can be found on the manufacturer's website at: [Logging Multimeter Benning MM 12](https://www.benning.de/products-en/testing-measuring-and-safety-equipment/digital-multimeter/logging-multimeter-mm-12.html).

**Caution:**  
In the technical specifications, Benning states a **basic accuracy** of only **0.5%** for the entire measuring device including all measuring functions. Due to the identical design with the [Appa 506B](https://www.appatech.com/en/product-553883/APPA-500-SERIES-MULTIMETERS-APPA-506-APPA-506B.html), this seems to be one of the worst values for AC voltage. The DC voltage measuring range, on the other hand, is given with a basic accuracy of **±(0.03% + 2 digit)**. Compare also with section 7.1 "Voltage ranges" in [Benning MM12 manual, 2020-02-19](https://www.benning.de/products-en/testing-measuring-and-safety-equipment/digital-multimeter/logging-multimeter-mm-12.html?file=files/benning/global_content/downloads/instruction_manuals/benning_mm12_manual.pdf&cid=41915).

![Voltage ranges and accuracy of Benning MM12 (source: Benning MM12 manual, 2020-02-19)](images/Benning_MM12_voltage_ranges_accuracy.png)  
Voltage ranges and accuracy of Benning MM12 (source: [Benning MM12 manual, 2020-02-19](https://www.benning.de/products-en/testing-measuring-and-safety-equipment/digital-multimeter/logging-multimeter-mm-12.html?file=files/benning/global_content/downloads/instruction_manuals/benning_mm12_manual.pdf&cid=41915))

## Communication with the Benning MM12

Communication with the Benning MM12 is done via Bluetooth or via the supplied infrared USB cable.

### Connection via Bluetooth

For communication via Bluetooth, the app *BENNING MM-CM Link* for Android and iOS is required. The Bluetooth connection is established independently via the app. This is unusual in that the Bluetooth connection is not managed via the central settings (e.g. in iOS) as is usual with other Bluetooth devices.

The app itself is kept very simple: a graphic display of the current measured values - but without a labeled timescale. It is possible to export the recorded data to CSV files.

Establishing a Bluetooth connection under Linux has so far been unsuccessful. The connection was always rejected by MM12 with the reference to incorrect authentication.

### Connection via Infrared-USB-Cable

![Top view of the Benning MM12 with IR connection port](images/Benning_MM12_top_view_small.jpeg)  
Top view of the Benning MM12 with IR connection port

The MM12 is supplied with the infrared USB cable [Appa IC-300U](https://sigrok.org/wiki/Device_cables#APPA_IC-300U) for plugging into the optical interface, shown in the following pictures:

![Infrared USB cable supplied with MM12](images/Benning_MM12_IR-USB-cable_small.jpeg)  
Infrared USB cable supplied with MM12

![Plug showing the IR LEDs of Infrared USB cable](images/Benning_MM12_IR-USB-cable_plug_small.jpeg)  
Plug showing the IR LEDs of Infrared USB cable

In **Windows** a driver for the USB to serial converter *[CP2102 USB to UART Bridge Controller](https://www.silabs.com/developers/usb-to-uart-bridge-vcp-drivers)* has to be installed.

Under **Linux** (e.g. Ubuntu or Raspberry Pi OS) the kernel module for the *CP2102* is already installed. After connecting the MM12 via the infrared USB cable a serial interface `/dev/ttyUSB0` or `/dev/ttyUSB1` is immediately available.

## Software

### Benning PC-Win MM 12

Benning supplies the Windows software *Benning PC-Win MM 12* on CD with the device. Alternatively it can be downloaded here: [Benning PC-Win MM 12 (Version 1.2.2) (170.8 MiB)](https://www.benning.de/products-en/testing-measuring-and-safety-equipment/digital-multimeter/logging-multimeter-mm-12.html?file=files/benning/global_content/downloads/software_and_terms_of_warranty/digital_multimeter/software_pc_win_benning_mm12.zip&cid=117242). 

The software itself is functionally reduced to the most necessary and essentially displays the current measured values graphically. A CSV export of the recorded data is also possible - however, the export function is extremely hidden and thus not intuitively accessible.

### Sigrok and Appa-Driver 

There are also open source projects that develop application software for the **Benning MM12** and other nearly identical devices such as the **Appa 506B** or the **Voltcraft VC-950**. Here, first of all, the project *[Sigrok](https://sigrok.org/wiki/Main_Page)* should be mentioned, whose aim is to develop a portable, cross-platform, Free/Libre/Open Source signal analysis software suite that supports various types of devices (e.g. multimeters, logic analysers, oscilloscopes and many more).

The modular structure of *Sigrok* makes it possible to constantly expand the software with new application modules as well as driver modules for an ever increasing number of measuring devices. So there is also a driver for the device series **Appa 500**, which should also support the **Benning MM12** at the same time. The following Sigrok Wiki pages are interesting for this:

- [Benning MM 12](https://sigrok.org/wiki/BENNING_MM_12)
- [Appa 500 Series](https://sigrok.org/wiki/APPA_500_Series)
- [Appa Multimeters](https://sigrok.org/wiki/APPA_Multimeters)

The Sigrok driver is not yet included in the mainline, as it still has the status "in progress" (as of 2022-05-15). So it has to be built manually from the Github sources: [Cymaphore /
libsigrok](https://github.com/Cymaphore/libsigrok).

Due to the many software dependencies to consider when compiling Sigrok and its modules (see here: [Sigrok Build requirements](http://sigrok.org/wiki/Building)) and the not yet stable development stage of the Appa driver, I have not yet succeeded in building a runnable version of Sigrok with the Appa driver enabled. The program `sigrok-cli` crashes with strange error messages ...

# Control via Python and PySerial

For communication with the MM12 a serial bytecode protocol is used, which is roughly based on the [Interface protocol of the Voltcraft VC-950](https://asset.conrad.com/media10/add/160267/c1/-/en/000124705DS01/datenblatt-124705-voltcraft-vc950-hand-multimeter-digital-datenlogger-cat-iii-1000-v-cat-iv-600-v-anzeige-counts-100000.pdf).

However, compared to the MM12 protocol there are considerable differences in the individual byte sequences of the commands and the evaluation of the byte sequences of the responses. Benning has not yet published a protocol description suitable for the MM12. A friendly email request to the [Technical Support](mailto:helpdesk@benning.de) helped. They immediately sent me the appropriate protocol documentation *Multimeter Benning MM 12 / Clamp Meter Benning CM 12-series Communication Protocol, Version 2.6, Status 2016-02-16*.

In the following implementation in Python I restrict myself exclusively to serial communication via the infrared USB interface. According to the documentation above, the serial protocol should also work for a communication via a serial Bluetooth interface. Unfortunately, I have not yet succeeded in establishing this Bluetooth connection to the Benning MM12.

## Direct communication via bytecode protocol

The following code sections show how to communicate directly with the MM12 using the bytecode protocol.

### Global imports and definitions

In [1]:
import serial
import time

SERIAL_PORT = "/dev/ttyUSB0"

MM12_READ_INFOS   = '55 55 00 00 AA'
MM12_READ_DISPLAY = '55 55 01 00 AB'

### Global functions

Define a function to convert byte strings to byte:

In [2]:
def func_convert_byteString_2_bytes(str_in):
    # remove whitespaces separating the hex bytes
    hex_str = str_in.replace(" ", "")

    hex_bytes = bytes.fromhex(hex_str)
    
    return hex_bytes

Define a function to convert hex strings to human readable ones:

In [3]:
def func_convert_hex_str_human_readable(str_in):
    hex_string = str_in.hex()
    # fill string with spaces for better reading of the hex string
    hex_string_wSpaces = " ".join(hex_string[i-1:i+1] for i, c in enumerate(hex_string) if i%2)
    return hex_string_wSpaces

Define a function to convert hex strings to ASCII:

In [4]:
def func_convert_hex2ascii(hex_str):
    # convert hex string to bytes object
    bytes_object = bytes.fromhex(hex_str)

    # convert bytes object to ASCII representation
    ascii_str = bytes_object.decode("ASCII")

    # replace unnecessary characters
    ascii_str = ascii_str.replace("\x00", "")
    # strip leading and trailing whitespaces
    ascii_str = ascii_str.strip()
    
    return ascii_str

Define a function to convert hex strings to integers:

In [5]:
def func_convert_hex2int(hex_str):
    # convert hex string to bytes object
    bytes_object = bytes.fromhex(hex_str)
    
    # convert hex bytes to integer
    out_int = int.from_bytes(bytes_object, 'little', signed=False)
    
    return out_int

### Open serial port to MM12

In [6]:
try:
    serial = serial.Serial(port=SERIAL_PORT, baudrate=9600, bytesize=8, parity='N', stopbits=1, timeout=2)

except serial.SerialException:
    print("Could not open serial port '{0}'!".format(SERIAL_PORT))
    raise

### Read device information

Send `Read information` command to MM12:

In [7]:
# flush input buffer, discarding all its contents
serial.reset_input_buffer()

# convert byte string of command to bytes
hex_bytes = func_convert_byteString_2_bytes(MM12_READ_INFOS)
ret = serial.write(hex_bytes)

hex_bytes_str = func_convert_hex_str_human_readable(hex_bytes)

print("Command '{}' written, consisting of {} bytes".format(hex_bytes_str, ret))

Command '55 55 00 00 aa' written, consisting of 5 bytes


Read back the 57 byte response of the MM12:

In [8]:
hex_string = serial.read(57)
#hex_string

print(func_convert_hex_str_human_readable(hex_string))

# convert byte string to hex string
hex_bytes = hex_string.hex()

55 55 00 34 42 45 4e 4e 49 4e 47 20 4d 4d 31 32 20 20 20 20 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 20 20 20 20 20 20 20 20 32 38 36 30 30 30 38 32 06 00 73 00 8f


Evaluate the **bytes 0..31** of the data payload as they represent the **model name**:

In [10]:
# bytes 0..31 from data payload represent the model name
model_name_hex = hex_bytes[8:72]

model_name_str = func_convert_hex2ascii(model_name_hex)

# init dictionary for storing the device information
dict_dmm_infos = {}

dict_dmm_infos['model'] = ('Model name', model_name_str)

print("{}: {}".format(dict_dmm_infos['model'][0], dict_dmm_infos['model'][1]))

Model name: BENNING MM12


Evaluate the **bytes 32..47** of the data payload as they represent the **serial number**:

In [11]:
# bytes 32..47 from data payload represent the serial number
serial_number_hex = hex_bytes[72:104]

serial_number_str = func_convert_hex2ascii(serial_number_hex)

dict_dmm_infos['serial'] = ('Serial number', serial_number_str)

print("{}: {}".format(dict_dmm_infos['serial'][0], dict_dmm_infos['serial'][1]))

Serial number: 28600082


Evaluate the **bytes 48..49** of the data payload as they represent the **model ID**:

In [12]:
# bytes 48..49 from data payload represent the model ID
model_ID_hex = hex_bytes[104:108]

model_ID_int = func_convert_hex2int(model_ID_hex)

dict_dmm_infos['id'] = ('Model ID', model_ID_int)

print("{}: {}".format(dict_dmm_infos['id'][0], dict_dmm_infos['id'][1]))

Model ID: 6


Evaluate the **bytes 50..51** of the data payload as they represent the **firmware version**:

In [13]:
# bytes 50..51 from data payload represent the firmware version
firmware_version_hex = hex_bytes[108:112]

firmware_version_int = func_convert_hex2int(firmware_version_hex)

dict_dmm_infos['fw'] = ('FW version', firmware_version_int/100)

print("{}: {}".format(dict_dmm_infos['fw'][0], dict_dmm_infos['fw'][1]))

FW version: 1.15


Print the dictionary of stored **device information**:

In [14]:
for key, val in dict_dmm_infos.items():
    print("{}:\t{}".format(val[0], val[1]))

Model name:	BENNING MM12
Serial number:	28600082
Model ID:	6
FW version:	1.15


### Read display information (measuring values and units)

Send `Read display` command to MM12:

In [15]:
# flush input buffer, discarding all its contents
serial.reset_input_buffer()

# convert byte string of command to bytes
hex_bytes = func_convert_byteString_2_bytes(MM12_READ_DISPLAY)
ret = serial.write(hex_bytes)

hex_bytes_str = func_convert_hex_str_human_readable(hex_bytes)

print("Command '{}' written, consisting of {} bytes".format(hex_bytes_str, ret))

Command '55 55 01 00 ab' written, consisting of 5 bytes


In [16]:
hex_string = serial.read(17)

print(func_convert_hex_str_human_readable(hex_string))

hex_bytes = hex_string.hex()

55 55 01 0c 0d 00 e2 00 00 91 00 00 00 70 00 00 a7


The evaluation of the individual byte sequences of the data payload is rather complex - therefore this simple example does without it.

The complete evaluation is done in the newly created wrapper class **Benning_MM12_Serial** in the Python file ``Benning_MM12_class.py``.

## Using the wrapper class **Benning_MM12_Serial**

The new wrapper class **Benning_MM12_Serial** in the Python file ```Benning_MM12_class.py``` implements the communication with the digital multimeter (DMM) **Benning MM12** in my home lab. The communication is done via the shipped infrared USB cable using a serial protocol in byte codes.

### Connect and disconnect to the MM12

In [17]:
# import wrapper class Benning_MM12_Serial from python file Benning_MM12_class.py
from Benning_MM12_class import Benning_MM12_Serial

In [18]:
SERIAL_PORT = "/dev/ttyUSB0"

# create new device object for the DMM Benning MM12
dmm_mm12 = Benning_MM12_Serial(port=SERIAL_PORT)

In [19]:
# read connection state to the Benning MM12
dmm_mm12.status

'Connected'

In [20]:
# read connection details
dmm_mm12.connected_with

'Benning MM12 over USB'

In [47]:
# close the connection to the Benning MM12
dmm_mm12.closeConnection()

In [50]:
# open the connection again
dmm_mm12.openConnection(port=SERIAL_PORT)

### Read device information 

In [21]:
# get device infos
dict_dmm_infos = dmm_mm12.getDeviceInfos()

for key, val in dict_dmm_infos.items():
    print("{}:\t{}".format(val[0], val[1]))

Model name:	BENNING MM12
Serial number:	28600082
Model ID:	6
FW version:	1.15


### Read display information (measuring values and units)

Get measurements converted in **SI base units**:

In [22]:
dict_dmm_measurement_base = dmm_mm12.getMeasurement_baseUnits()

if len(dict_dmm_measurement_base) != 0:
    print("Display value in mode '{}': {} {}".format(dict_dmm_measurement_base['function'], dict_dmm_measurement_base['value'], dict_dmm_measurement_base['unit']))

Display value in mode '°C': 22.6 °C


Get measurements converted in **human readable units**:

In [23]:
dict_dmm_measurement_human = dmm_mm12.getMeasurement_humanUnits()

if len(dict_dmm_measurement_human) != 0:
    print("Display value: {} {}".format(dict_dmm_measurement_human['value'], dict_dmm_measurement_human['unit']))

Display value: 22.6 °C
